# NER

阅读材料:

https://download.csdn.net/blog/column/10841550/129149699

In [1]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [2]:
import os
from datasets import load_dataset

token = os.environ.get("HF_TOKEN")

dataset = load_dataset('peoples_daily_ner',token=token)

dataset

d:\Middlewares\anaconda3\envs\NLP2\lib\site-packages\datasets\load.py:1429: FutureWarning: The repository for peoples_daily_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/peoples_daily_ner
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt6")

print(tokenizer)

BertTokenizerFast(name_or_path='hfl/rbt6', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [4]:
tokenizer.batch_encode_plus(
    [
        [
            "海",
            "钓",
            "比",
            "赛",
            "地",
            "点",
            "在",
            "厦",
            "门",
            "与",
            "金",
            "门",
            "之",
            "间",
            "的",
            "海",
            "域",
            "。",
        ],
        [
            "这",
            "座",
            "依",
            "山",
            "傍",
            "水",
            "的",
            "博",
            "物",
            "馆",
            "由",
            "国",
            "内",
            "一",
            "流",
            "的",
            "设",
            "计",
            "师",
            "主",
            "持",
            "设",
            "计",
            "，",
            "整",
            "个",
            "建",
            "筑",
            "群",
            "精",
            "美",
            "而",
            "恢",
            "宏",
            "。",
        ],
    ],
    truncation=True,
    padding=True,
    return_tensors="pt",
    is_split_into_words=True,
)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[ 101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305,  680, 7032,
         7305,  722, 7313, 4638, 3862, 1818,  511,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [ 101, 6821, 2429,  898, 2255,  988, 3717, 4638, 1300, 4289, 7667, 4507,
         1744, 1079,  671, 3837, 4638, 6392, 6369, 2360,  712, 2898, 6392, 6369,
         8024, 3146,  702, 2456, 5029, 5408, 5125, 5401, 5445, 2612, 2131,  511,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [5]:
import torch
from datasets import load_dataset, load_from_disk


class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        #names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
        # 如果实体是个3字人名，应该表现为 122；

        #在线加载数据集
        dataset = load_dataset(path='peoples_daily_ner', split=split)
        
        #离线加载数据集
#         data_path = '/Users/xx/Downloads/NER_in_Chinese-main/data'
        #dataset = load_from_disk(dataset_path=data_path)[split]

        #过滤掉太长的句子
        def f(data):
            return len(data['tokens']) <= 512 - 2

        dataset = dataset.filter(f)

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        tokens = self.dataset[i]['tokens']
        labels = self.dataset[i]['ner_tags']

        return tokens, labels


dataset = Dataset('train')

tokens, labels = dataset[0]

# 查看数据样例
# 有两万多条数据，
len(dataset), tokens, labels

(20852,
 ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0])

In [6]:
# 数据整理函数
def collate_fn(data):
    tokens = [i[0] for i in data]
    labels = [i[1] for i in data]

    inputs = tokenizer.batch_encode_plus(tokens,
                                         truncation=True,
                                         padding=True,
                                         return_tensors='pt',
                                         is_split_into_words=True)
    

    # 找到最长的句子 长度
    lens = inputs['input_ids'].shape[1]
    
    # 对句子进行补充，头部/尾部+7，然后截取到最长的长度； 7不存在于
    for i in range(len(labels)):
        labels[i] = [7] + labels[i]
        labels[i] += [7] * lens
        labels[i] = labels[i][:lens]

    return inputs, torch.LongTensor(labels)


# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True,pin_memory=True)

In [7]:
# 查看数据样例
for i, (inputs, labels) in enumerate(loader):
    break

print(len(loader)) # 
print(tokenizer.decode(inputs['input_ids'][0]))
print(labels[0])

for k, v in inputs.items():
    print(k, v.shape)

1303
[CLS] 至 少 在 1 5 米 以 外 的 、 来 自 尼 日 利 亚 的 裁 判 鲍 查 多 略 犹 豫 一 下 ， 鸣 笛 判 罚 点 球 ， 从 而 救 了 意 大 利 队 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
tensor([7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 0, 0, 0, 1, 2, 2, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 0, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
input_ids torch.Size([16, 85])
token_type_ids torch.Size([16, 85])
attention_mask torch.Size([16, 85])


In [24]:
from transformers import AutoModel

# 加载预训练模型
pretrained = AutoModel.from_pretrained('hfl/rbt6')

pretrained = pretrained.to(device)

# 统计参数量 6kw 
print(sum(i.numel() for i in pretrained.parameters()) / 10000)

# 模型试算
# [b, lens] -> [b, lens, 768]
inputs = inputs.to(device)
pretrained(**inputs).last_hidden_state.shape

5974.0416


torch.Size([16, 85, 768])

In [42]:
# 定义下游模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = None  # 默认情况下，让预训练模型 并不属于下游任务模型的一部分；
        
        # 这里定义下游模型只有简单两层网络 RNN(GRU)+全连接
        self.rnn = torch.nn.GRU(768, 768,batch_first=True)
        self.fc = torch.nn.Linear(768, 8)

    def forward(self, inputs):
        
        inputs = inputs.to(device)
        if self.tuneing:  # 如果是 tuneing模式，预训练模型属于自己模型的一部分，使用自己的预训练模型进行计算 
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():  # 使用外部的预训练模型进行计算
                out = pretrained(**inputs).last_hidden_state

        out, _ = self.rnn(out)

        out = self.fc(out).softmax(dim=2)

        return out
    
    # 切换模型是否 tuneing 的模式 
    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in pretrained.parameters():
                i.requires_grad = True  # 预训练模型也进行参数更新，计算参数梯度 

            pretrained.train() # 预训练模型也是训练模式 
            self.pretrained = pretrained
        else:
            for i in pretrained.parameters():
                i.requires_grad_(False)

            pretrained.eval() # 运行模式，不梯度更新
            self.pretrained = None


model = Model()

model.to(device)

model(inputs).shape # torch.Size([16, 8, 8])

torch.Size([16, 85, 8])

In [29]:
# 1、对计算结果和label变形,并且移除pad
def reshape_and_remove_pad(outs, labels, attention_mask):
    #变形,便于计算loss
    #[b, lens, 8] -> [b*lens, 8]
    outs = outs.reshape(-1, 8)
    #[b, lens] -> [b*lens]
    labels = labels.reshape(-1)

    #忽略对pad的计算结果
    #[b, lens] -> [b*lens - pad]
    select = attention_mask.reshape(-1) == 1
    outs = outs[select]
    labels = labels[select]

    return outs, labels


reshape_and_remove_pad(torch.randn(2, 3, 8), torch.ones(2, 3),
                       torch.ones(2, 3))

(tensor([[ 0.4650, -0.7309,  0.0480, -1.2878, -0.4159,  0.3020,  1.3637,  0.2327],
         [ 1.2281,  0.6102, -0.6525,  1.0780,  1.7088,  0.6285,  0.0045,  0.9218],
         [-0.6620, -0.0954, -0.5760,  0.3274,  0.1450, -1.5258,  1.6959,  1.1467],
         [ 0.0093, -0.3787,  0.3565, -0.3704, -1.1949, -0.4401,  0.9939,  1.5427],
         [-0.8264,  0.8301, -0.6192,  1.0794,  0.8094, -1.8064, -1.0796, -1.5944],
         [ 0.0753,  0.8013, -0.1799,  0.3314,  0.4346,  1.5185, -1.2893, -0.4233]]),
 tensor([1., 1., 1., 1., 1., 1.]))

In [11]:
def get_correct_and_total_count(labels, outs):
    #[b*lens, 8] -> [b*lens]
    outs = outs.argmax(dim=1)
    correct = (outs == labels).sum().item()
    total = len(labels)

    #计算除了0以外元素的正确率,因为0太多了,包括的话,正确率很容易虚高
    select = labels != 0
    outs = outs[select]
    labels = labels[select]
    correct_content = (outs == labels).sum().item()
    total_content = len(labels)

    return correct, total, correct_content, total_content


get_correct_and_total_count(torch.ones(16), torch.randn(16, 8))

(3, 16, 3, 16)

In [36]:
from transformers import AdamW

model_path='./model/ner.model'
 
def train(epochs):
    lr = 2e-5 if model.tuneing else 5e-4

    #训练
    optimizer = AdamW(model.parameters(), lr=lr)
    #optimizer = optimizer.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    criterion = criterion.to(device)

    model.train()
    for epoch in range(epochs):
        for step, (inputs, labels) in enumerate(loader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            #模型计算
            #[b, lens] -> [b, lens, 8]
            outs = model(inputs)
            outs = outs.to(device)
            

            #对outs和label变形,并且移除pad
            #outs -> [b, lens, 8] -> [c, 8]
            #labels -> [b, lens] -> [c]
            outs, labels = reshape_and_remove_pad(outs, labels,
                                                  inputs['attention_mask'])
            
            outs,labels = outs.to(device),labels.to(device)

            #梯度下降
            loss = criterion(outs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if step % 50 == 0: # 计算正确数
                counts = get_correct_and_total_count(labels, outs)

                accuracy = counts[0] / counts[1] 
                accuracy_content = counts[2] / counts[3]

                print('-- ', epoch, step, loss.item(), accuracy, accuracy_content)
        
        
        torch.save(model, model_path)

In [30]:
model.fine_tuneing(False)
model = model.to(device)

for name,param in model.named_parameters():
    print(f"Parameter: {name}, Device: {param.device}")

Parameter: rnn.weight_ih_l0, Device: cuda:0
Parameter: rnn.weight_hh_l0, Device: cuda:0
Parameter: rnn.bias_ih_l0, Device: cuda:0
Parameter: rnn.bias_hh_l0, Device: cuda:0
Parameter: fc.weight, Device: cuda:0
Parameter: fc.bias, Device: cuda:0


In [19]:
print(model.pretrained)

None


In [37]:
# 非 fine-tuneing 训练一个轮次

model = model.to(device)
model.fine_tuneing(False)
#print(sum(p.numel() for p in model.parameters()) / 10000)
train(1)  # 354 万个参数

# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Device: {param.device}")

--  0 0 1.3471088409423828 0.9269005847953217 0.3902439024390244
--  0 50 1.3740086555480957 0.9 0.3018867924528302
--  0 100 1.3917919397354126 0.8822170900692841 0.23880597014925373
--  0 150 1.334036111831665 0.9399727148703957 0.42105263157894735
--  0 200 1.363770604133606 0.9102384291725105 0.3333333333333333
--  0 250 1.380600929260254 0.8934081346423562 0.2962962962962963
--  0 300 1.395739197731018 0.8782696177062375 0.20915032679738563
--  0 350 1.325000524520874 0.9490084985835694 0.47058823529411764
--  0 400 1.3658952713012695 0.9081145584725537 0.29357798165137616
--  0 450 1.3706334829330444 0.9033760186263097 0.2782608695652174
--  0 500 1.3781756162643433 0.8958333333333334 0.26229508196721313
--  0 550 1.37593412399292 0.8980747451868629 0.26229508196721313
--  0 600 1.394051432609558 0.8799582463465553 0.21768707482993196
--  0 650 1.4532809257507324 0.8207282913165266 0.2
--  0 700 1.4029980897903442 0.8710106382978723 0.24806201550387597
--  0 750 1.386158227920532

In [44]:
model.fine_tuneing(True)
print(sum(p.numel() for p in model.parameters()) / 10000)
print(model.pretrained)
train(2) # 6000多万个参数

6329.012
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fa

In [2]:
import torch
model_path='./model/ner.model'
def predict():
    model_load = torch.load(model_path)
    model_load.eval()

    loader_test = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (inputs, labels) in enumerate(loader_test):
        break

    with torch.no_grad():
        #[b, lens] -> [b, lens, 8] -> [b, lens]
        outs = model_load(inputs).argmax(dim=2)

    for i in range(32):
        #移除pad
        select = inputs['attention_mask'][i] == 1
        input_id = inputs['input_ids'][i, select]
        out = outs[i, select]
        label = labels[i, select]
        
        #输出原句子
        print(tokenizer.decode(input_id).replace(' ', ''))

        #输出tag
        for tag in [label, out]:
            s = ''
            for j in range(len(tag)):
                if tag[j] == 0:
                    s += '·'
                    continue
                s += tokenizer.decode(input_id[j])
                s += str(tag[j].item())

            print(s)
        print('==========================')


predict()

NameError: name 'model_path' is not defined